# Optimization with Constraint Learning: A WFP case study

## The Palatable Diet Problem  

<font size="4"> Problem Description</font>    

In this case study, we focus on the diet problem related to [(Peters et al., 2021)](https://pubsonline.informs.org/doi/10.1287/ijoo.2019.0047) which seeks to optimize humanitarian food aid. The model formulated by [Peters et al. (2021)](https://pubsonline.informs.org/doi/10.1287/ijoo.2019.0047) aims to provide the World Food Programme (WFP) with a decision-making tool for long-term recovery operations, which simultaneously optimizes the food basket to be delivered, the sourcing plan, the delivery plan, and the transfer modality of a month-long food supply. The model proposed by [Peters et al. (2021)](https://pubsonline.informs.org/doi/10.1287/ijoo.2019.0047) enforces that the food baskets address the nutrient gap and are palatable. To guarantee a certain level of palatability, the authors use a number of “unwritten rules” that have been defined in collaboration with nutrition experts. 

<div>
<img src="figures/supplychain_0.jpg" width="400"/>
</div>

In this case study, we restrict the model to the diet problem and we take a step further by inferring palatability constraints directly from data that reflects local people's opinions. The conceptual model presents a linear optimization (LO) structure with only the food palatability constraint to be learned. Data on palatability is generated through a simulator, but the procedure would remain unchanged if data were collected in the field, for example through surveys. The structure of this problem, which is an LO and involves only one learned constraint, allows the following analyses: (1) the effect of the trust-region on the optimal solution, and (2) the effect of multiple learned models to define the same constraint. 

In [1]:
import pandas as pd
import numpy as np
import math
import time
import os
import warnings

# Optimization with Constraint Leagning package
import opticl

# Optimization modelling
from pyomo import environ
from pyomo.environ import *

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
np.random.seed(0)

### Food Basket Palatability  
Based on [(Peters et al., 2021)](https://pubsonline.informs.org/doi/10.1287/ijoo.2019.0047), a food basket is defined by 25 foods (e.g., beans, meat, oil) and their relative amount in grams. Each food belongs to one of the five macro-categories: cereals and grains, pulses and vegetables, oils and fats, mixed and blended foods, meat and fish and dairy. For each macro-category $g \in \mathcal{G}$ an upper and lower bound are defined, respectively $max_g$ and $min_g$. While in [(Peters et al., 2021)](https://pubsonline.informs.org/doi/10.1287/ijoo.2019.0047), the authors use bound constraints to ensure the food basket palatability, we extend the definition of palatability by mean of a (non-negative) palatability score that is closer to zero for more palatable diets. The score is defined as  
\begin{align}
Palatability\ Score = \sqrt{\sum_{g\in \mathcal{G}}(\gamma_{g}(\widehat{x}_{g}-Opt_{g}))^{2}},\label{eqn:palatability_score}
\end{align}
where
$$ \widehat{x}_{g} = \sum_{k \in \mathcal{K}_g} x_{k} ~~ \text{with} \ g \in \mathcal{G} \text{ and}$$
$$ Opt_{g} = \frac{max_{g} + min_{g}}{2} ~~ \text{with} \ g \in \mathcal{G}.
$$

Since different macro-categories have different range sizes ($max_g - min_g$), a parameter $\gamma_{g}$ is used to scale their impact on the score, see Table 1. 

<br/>
<div>
<img src="figures/table.jpg" width="400"/>
</div>

The palatability score is afterwards normalized in a [0,1] interval and subtracted to 1 in order to obtain a value closer to 1 for more palatable diets.

In [3]:
def normalize(y):
    # Values based on the dataset before normalization
    minimum = 71.969 
    maximum = 444.847  
    return 1 - (y - minimum)/(maximum - minimum)

In [4]:
def check_violation(threshold, solution):
    # Cereals & Grains
    group1 = [1, 11, 12, 14, 15, 22, 23]
    group1_names = [list(solution.keys())[i] for i in group1]
    values1 = [solution[x] for x in group1_names]
    food_in_group1 = sum(values1)*100
    idealG1 = 400
    distG1 = food_in_group1 - idealG1
    
    # Pulses & Vegetables
    group2 = [0, 6, 10, 13]
    group2_names = [list(solution.keys())[i] for i in group2]
    values2 = [solution[x] for x in group2_names]
    food_in_group2 = sum(values2)*100
    idealG2 = 65
    distG2 = food_in_group2 - idealG2
    
    # Oils & Fats
    group3 = [21]
    group3_names = [list(solution.keys())[i] for i in group3]
    values3 = [solution[x] for x in group3_names]
    food_in_group3 = sum(values3)*100
    idealG3 = 27.5
    distG3 = food_in_group3 - idealG3
    
    # Mixed & Blended Foods
    group4 = [5, 24, 16, 17, 18, 19]
    group4_names = [list(solution.keys())[i] for i in group4]
    values4 = [solution[x] for x in group4_names]
    food_in_group4 = sum(values4)*100
    idealG4 = 45
    distG4 = food_in_group4 - idealG4
    
    # Meat & Fish & Dairy
    group5 = [2, 3, 4, 7, 8]
    group5_names = [list(solution.keys())[i] for i in group5]
    values5 = [solution[x] for x in group5_names]
    food_in_group5 = sum(values5)*100
    idealG5 = 30
    distG5 = food_in_group5 - idealG5
    
    real_palatability = np.round(math.sqrt(distG1 ** 2 + (5.7 * distG2) ** 2 + (16 * distG3) ** 2
                                      + (4.4 * distG4) ** 2 + (6.6 * distG5) ** 2), 3)
    real_palatability_norm = normalize(real_palatability)
    return 1-int(real_palatability_norm>=threshold), real_palatability_norm

## Step 1: Conceptual model  

**Objective function:** minimize the total cost of the food basket.  
\begin{align}\min_{\boldsymbol{x}, y} \boldsymbol{c}^\top \boldsymbol{x}\end{align}

*subject to* 

**Nutritional constraints:** for each nutrient $j\in\mathcal{N}$, at least meet the minimum required level.  
\begin{align} \sum_{k \in \mathcal{K}} nutval_{kj} x_{k} \geq nutreq_{j}, \ \ \ \forall l\in\mathcal{N},\end{align}
**Constraints on sugar and salt:**</font>  
\begin{align} x_{salt} = 5,\end{align}   
\begin{align} x_{sugar} = 20,\end{align} 
<span style="color:blue">**Palatability constraints:**</span> the food basket palatability has to be at least equal to $t$.  
\begin{align} y \geq t,\end{align}  
<span style="color:blue">**Learned predictive model:**</span>. the palatability is defined using a predictive model.  
\begin{align} y = \hat{h}(\boldsymbol{x}),\end{align} 
**Non negativity constraints:**  
\begin{align} x_{k} \geq 0, \ \ \ \forall k \in \mathcal{K}.\end{align}

In [5]:
def init_conceptual_model(cost_p):
    N = list(nutr_val.index)  # foods
    M = nutr_req.columns  # nutrient requirements
    model = ConcreteModel('TPDP')
    '''
    Decision variables
    '''
    model.x = Var(N, domain=NonNegativeReals)  # variables controlling the food basket
    '''
    Objective function.
    '''
    def obj_function(model):
        return sum(cost_p[food].item()*model.x[food] for food in N)
    model.OBJ = Objective(rule=obj_function, sense=minimize)
    '''
    Nutrients requirements constraint.
    '''
    def constraint_rule1(model, req):
        return sum(model.x[food] * nutr_val.loc[food, req] for food in N) >= nutr_req[req].item()
    model.Constraint1 = Constraint(M, rule=constraint_rule1)
    '''
    Sugar constraint
    '''
    def constraint_rule2(model):
        return model.x['Sugar'] == 0.2
    model.Constraint2 = Constraint(rule=constraint_rule2)
    '''
    Salt constraint
    '''
    def constraint_rule3(model):
        return model.x['Salt'] == 0.05
    model.Constraint3 = Constraint(rule=constraint_rule3)
    return model

## Step 2: Learn the *palatability* constraints

### Data Loading

In [6]:
nutr_val = pd.read_excel('processed-data/Syria_instance.xlsx', sheet_name='nutr_val', index_col='Food')
nutr_req = pd.read_excel('processed-data/Syria_instance.xlsx', sheet_name='nutr_req', index_col='Type')
cost_p = pd.read_excel('processed-data/Syria_instance.xlsx', sheet_name='FoodCost', index_col='Supplier').iloc[0,:]
dataset = pd.read_csv('processed-data/WFP_dataset.csv').sample(frac=1)

y = dataset['label']
X = dataset.drop(['label'], axis=1, inplace=False)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
dataset

,Beans,Bulgur,Cheese,Fish,Meat,CSB,Dates,DSM,Milk,Salt,...,Soya-fortified bulgur wheat,Soya-fortified maize meal,Soya-fortified sorghum grits,Soya-fortified wheat flour,Sugar,Oil,Wheat,Wheat flour,WSB,label
398,0.687675,1.257354,0.000000,0.0,0.0,0.000000,0.000000,0.302104,0.000000,0.05,...,0.0,0.000000,0.0,0.0,0.2,0.357429,2.823603,0.000000,0.637964,0.715428
3833,0.551125,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.117990,0.000000,0.05,...,0.0,0.000000,0.0,0.0,0.2,0.392274,2.540599,3.414615,0.733300,0.292719
4836,0.701614,0.000000,0.000000,0.0,0.0,0.094990,0.000000,0.330808,0.000000,0.05,...,0.0,0.000000,0.0,0.0,0.2,0.221908,0.336647,0.000000,0.545864,0.816616
4572,0.000000,3.832166,0.000000,0.0,0.0,0.000000,0.626751,0.278648,0.132718,0.05,...,0.0,0.000000,0.0,0.0,0.2,0.311117,0.000000,0.000000,0.694007,0.794680
636,0.039754,0.000000,0.344293,0.0,0.0,0.000000,0.000000,0.106482,0.000000,0.05,...,0.0,0.000000,0.0,0.0,0.2,0.160220,0.000000,0.000000,0.788790,0.261417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4931,0.743757,0.000000,0.115426,0.0,0.0,0.700000,0.000000,0.479955,0.000000,0.05,...,0.0,0.041521,0.0,0.0,0.2,0.181913,0.000000,0.000000,0.000000,0.210851
3264,0.779749,0.000000,0.075277,0.0,0.0,0.761232,0.000000,0.392905,0.000000,0.05,...,0.0,0.000000,0.0,0.0,0.2,0.178567,0.000000,0.000000,0.029013,0.293570
1653,0.668713,3.322546,0.000000,0.0,0.0,0.429041,0.000000,0.450216,0.000000,0.05,...,0.0,0.000000,0.0,0.0,0.2,0.271382,0.000000,0.000000,0.189249,0.813417
2607,0.460621,1.464405,0.160834,0.0,0.0,0.757835,0.000000,0.391194,0.000000,0.05,...,0.0,0.000000,0.0,0.0,0.2,0.166244,0.000000,0.000000,0.000000,0.382687


### Training

In [37]:
alg_dict = {'linear': None, 'svm': None, 'cart': None, 'rf': None, 'gbm': None, 'mlp': None}

viol_rule = 0.5 # 'average'

gr=False
bs = 0

print("Algorithms = %s" % alg_dict)
print("Bootstrap iterations = %d" % bs)
print("Violation rule = %s" % str(viol_rule))
code_version = 'AAAI-23_WFPexample'

version = 'vAAAI-23_WFPexample'
outcome = 'palatability'
threshold = 0.5

Algorithms = {'gbm': None}
Bootstrap iterations = 0
Violation rule = 0.5


In [44]:
data = X_train
outcome_list = {'palatability': {'lb':threshold, 'ub':None, 'objective_weight':0,'group_models':gr,
'task_type': 'continuous', 'alg_list':alg_dict, 'bootstrap_iterations':bs,
                                   'X_train':X_train, 'y_train':y_train, 'X_test':X_test, 'y_test':y_test,
                                   'dataset_path':'processed-data/WFP_dataset.csv'}}

performance = opticl.train_ml_models(outcome_list, version)
if not os.path.exists('results'):
    os.makedirs('results')
performance.to_csv('results/%s_performance.csv' % (code_version))

columns_df = ['algorithm','iteration','price_matrix']+list(X.columns)+['objective_function', 'real_palat', 'pred_palat', 'violation', 'time']

print("\nPreparing model master")
if viol_rule == 'average':
    gr_method = 'average'
    max_viol = None
    print("Group method = %s" % (gr_method))
    gr_string = 'average'
else: 
    gr_method = 'violation'
    max_viol = float(viol_rule)
    print("Group method = %s (violation limit = %.2f)" % (gr_method, max_viol))
    gr_string = 'violation_%.2f' % max_viol

Learning a model for palatability
No bootstrap - training on full training data
training palatability with gbm
------------- Initialize grid  ----------------
------------- Running model  ----------------
Algorithm = gbm, metric = None
saving... results/gbm_palatability_trained.pkl
------------- Model evaluation  ----------------
-------------------training evaluation-----------------------
Train MSE: 0.004570455119563544
Train R2: 0.910193525532487
-------------------testing evaluation-----------------------
Test MSE: 0.00595551970521792
Test R2: 0.8855012914122791

Saving the performance...
Done!

Preparing model master
Group method = violation (violation limit = 0.50)


Train and validation scores: roc_auc (binary classification), neg_mean_squared_error (regression)

In [ ]:
performance

### Select fitted models   
Select models using aggregated 'performance' table. The models are selected based on the highest *valid_score*, assuming higher scores are better

In [39]:
mm = opticl.initialize_model_master(outcome_list)
mm.loc[outcome,'group_method'] = gr_method
mm.loc[outcome,'max_violation'] = max_viol
mm.loc[outcome, 'trust_region'] = True
model_master = opticl.model_selection(mm, performance)

if not os.path.exists('experiments'):
    print('Creating folder...')
    os.makedirs('experiments')
model_master.to_csv('experiments/model_master_%s.csv' % (code_version), index = True)

opticl.check_model_master(model_master)
model_master

Checking model
No learned objective
1 learned constrained outcomes

Checking outcome palatability
Embedding outcome with single cart model

Embedding constraint for palatability.
0.5 <= palatability


,model,task,objective,lb,ub,features,var_features,contex_features,group_models,group_method,ensemble_weights,max_violation,trust_region,dataset_path,clustering_model,enlargement,SCM_counterfactuals
palatability,{'results/cart/vAAAI-23_WFPexample_palatabilit...,continuous,0,0.5,None,"Index(['Beans', 'Bulgur', 'Cheese', 'Fish', 'M...","Index(['Beans', 'Bulgur', 'Cheese', 'Fish', 'M...",{},False,violation,None,0.2,True,processed-data/WFP_dataset.csv,None,[0],None


## Step 3: Solve the optimization problem

In [42]:
start_cm = time.time()
conceptual_model= init_conceptual_model(cost_p)
cm = time.time() - start_cm

start_opticl = time.time()
final_model = opticl.optimization_MIP(conceptual_model, model_master)
model_opticl = time.time() - start_opticl
# final_model.write('experiments/mip_%s_seed_%d.lp' % (code_version, seed))
opt = SolverFactory('glpk')
start_time = time.time()
results = opt.solve(final_model, logfile = "WFP_logfile.txt", tee=True) 
computation_time = time.time() - start_time

Generating constraints for the trust region using 5000 samples.
... Trust region defined.
Embedding constraints for palatability
Adding single model.
GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\dmaragn\AppData\Local\Temp\tmpub33fhvk.glpk.raw --wglp C:\Users\dmaragn\AppData\Local\Temp\tmpufo20_1v.glpk.glp
 --cpxlp C:\Users\dmaragn\AppData\Local\Temp\tmpwtc2pz1q.pyomo.lp
Reading problem data from 'C:\Users\dmaragn\AppData\Local\Temp\tmpwtc2pz1q.pyomo.lp'...
C:\Users\dmaragn\AppData\Local\Temp\tmpwtc2pz1q.pyomo.lp:56007: warning: lower bound of variable 'x27' redefined
C:\Users\dmaragn\AppData\Local\Temp\tmpwtc2pz1q.pyomo.lp:56007: warning: upper bound of variable 'x27' redefined
263 rows, 5063 columns, 50122 non-zeros
36 integer variables, all of which are binary
56043 lines were read
Writing problem data to 'C:\Users\dmaragn\AppData\Local\Temp\tmpufo20_1v.glpk.glp'...
60740 lines were written
GLPK Integer Optimizer, v4.65
263 rows, 506

### Evaluation

In [19]:
def evaluation(model_eval, cm, model_opticl, computation_time):
    pred_palat = value(model_eval.y[outcome])
    violation_bool, real_palat = check_violation(threshold,  model_eval.x.get_values())
    ## Save solutions
    solution = {food: str(np.round(model_eval.x.get_values()[food]*100, 2))+' g' for food in model_eval.x.get_values().keys() if np.round(model_eval.x.get_values()[food],2) > 0}

    print('\n################################### Summary ###################################')
    print(f'Optimal Solution: {solution}\n')
    print("Total cost: %.3f $" % value(model_eval.OBJ))
    print("Predicted palatability: %.3f" % pred_palat)
    print("Real palatability: %.3f" % real_palat)

    print(f'\nConceptual model defined in {np.round(cm, 3)} seconds')
    if model_master.loc['palatability', 'trust_region']:
        print(f'Learned constraints & trust region embedded in {np.round(model_opticl, 3)} seconds')
    else:
        print(f'Learned constraints embedded in {np.round(model_opticl, 3)} seconds')
    print(f'Problem Solved in {np.round(computation_time, 3)} seconds')
    print('###############################################################################')

In [43]:
evaluation(final_model, cm, model_opticl, computation_time)


################################### Summary ###################################
Optimal Solution: {'Beans': '48.13 g', 'DSM': '1.35 g', 'Milk': '48.09 g', 'Salt': '5.0 g', 'Lentils': '7.69 g', 'Maize': '39.82 g', 'Maize meal': '200.46 g', 'Soya-fortified bulgur wheat': '1.51 g', 'Sugar': '20.0 g', 'Oil': '26.48 g', 'Wheat': '105.45 g', 'WSB': '70.0 g'}

Total cost: 3509.145 $
Predicted palatability: 0.655
Real palatability: 0.683

Conceptual model defined in 0.005 seconds
Learned constraints & trust region embedded in 1.392 seconds
Problem Solved in 0.784 seconds
###############################################################################


## Trust region  

As the optimal solutions of optimization problems are often at the extremes of the feasible region, this can be problematic for the validity of the trained ML model. Generally speaking the accuracy of a predictive model deteriorates for points that are further away from the data points in $\mathcal{D}$ [(Goodfellow et al. 2015)](https://arxiv.org/abs/1412.6572). To mitigate this problem, we elaborate on the idea proposed by [Biggs et al. (2021)](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2986630) to use the convex hull (CH) of the dataset as a *trust region* constraint to prevent the predictive model from extrapolating. If $\boldsymbol{X} = \{ \boldsymbol{\hat{x}}_i \}_{i=1}^N$ is the set of observed input data with $\boldsymbol{\hat{x}}_i = (\boldsymbol{\bar{x}}_i, \boldsymbol{\bar{w}}_i)$, we define the trust region as the CH of this set and denote it by CH($\boldsymbol{X}$). Recall that CH($\boldsymbol{X}$) is the smallest convex polytope that contains the set of points $\boldsymbol{X}$. It is well-known that computing the CH is exponential in time and space with respect to the number of samples and their dimensionality. However, since the CH is a polytope, explicit expressions for its facets are not necessary. More precisely, CH($\boldsymbol{X}$) is represented as
\begin{align}
    \text{CH($\boldsymbol{X}$)} = \bigg\{ \boldsymbol{x} \bigg| \sum_{i \in \mathcal{I}} \lambda_i \boldsymbol{\hat{x}}_i = \boldsymbol{x}, \ \sum_{i \in \mathcal{I}} \lambda_i = 1, \ \boldsymbol{\lambda} \geq 0
    \bigg\},
\label{eqn:trust_region1}
\end{align}
where $\boldsymbol{\lambda} \in \mathbb{R}^N$, and $\mathcal{I} = \{1, \dots, N \}$ is the index set of samples in 
$\boldsymbol{X}$.

<br>
<font size="4">Extensions</font> 

**1) Clustering**  
In situations such as the left figure in:
<div>
<img src="figures/CTR.jpg" width="9000"/>
</div>
CH($\boldsymbol{X}$) includes regions with few or no data points (low-density regions). Blindly using CH($\boldsymbol{X}$) in this case can be problematic if the solutions are found in the low-density regions. We therefore advocate the use of a two-step approach. First, clustering is used to identify distinct high-density regions, and then the trust region is represented as the union of the CHs of the individual clusters. The trust region formulation becomes:

\begin{align}
    \bigcup_{k\in\mathcal{K}}\text{CH($\boldsymbol{X}_k$)} = \bigg\{ \boldsymbol{x}
    \bigg| \sum_{i \in \mathcal{I}_k} 
    \lambda_i \boldsymbol{\hat{x}}_i = \boldsymbol{x}, \ \sum_{i \in \mathcal{I}_k}  \lambda_i = u_k \ \forall k \in \mathcal{K}, \sum_{k \in \mathcal{K}} u_k = 1, \ \boldsymbol{\lambda} \geq 0, \ \boldsymbol{u} \in \{0,1\}^{|\mathcal{K}|}
    \bigg\},
\label{eqn:trust_region2}
\end{align}
where $\mathcal{K}$ is the set of clusters.  

**2) Column selection**  
The number of variables used to define the trust region increases with the increase in the dataset size, which may make the optimization process prohibitive when the number of samples becomes too large. We therefore provide a **column selection algorithm** that selects a small subset of the samples. This algorithm can be directly used with convex optimization problems or embedded as part of a branch and bound algorithm when the optimization problem involves integer variables.
<div>
<img src="figures/CSTR.jpg" width="1000"/>
</div>

**3) Enlarged Trust region**  
Although we introduced the trust region as a set of constraints to preserve the predictive performance of the fitted constraints, in their recent paper, [Balestriero et al. (2021)](https://arxiv.org/abs/2110.09485) show how likely is to extrapolate in a high-dimensional dataset and therefore, how the generalization performance is typically obtained using samples outside the interpolation region. In light of this evidence, we propose an $\epsilon$-CH formulation which enables the optimal solution to be outside $CH(X)$. The $\epsilon$-CH is formulated as follows:
\begin{align}
\text{$\epsilon$-CH($\boldsymbol{X}$)} = \bigg\{ (\boldsymbol{x},\boldsymbol{s}) \bigg| \sum_{i \in \mathcal{I}} \lambda_i \boldsymbol{\hat{x}}_i = \boldsymbol{x} + \boldsymbol{s}, \ \sum_{i \in \mathcal{I}} \lambda_i = 1, \ \boldsymbol{\lambda} \geq 0, \ ||\boldsymbol{s}||_{p} \leq \epsilon
    \bigg\},
\label{eqn:epstrust_region1} 
\end{align}

<div>
<img src="figures/ETR.jpg" width="1000"/>
</div>


#### Enlarged trust region
[x, , ]-> 0: no enlargement, 1: enlargement  
[ ,x, ]-> 0: enlargement using a bounding constraint, 1: enlargement using a penalty term in the objective function  
[ , ,x]-> penalty coefficient/upper bound

In [24]:
mm = opticl.initialize_model_master(outcome_list)
mm.loc[outcome,'group_method'] = gr_method
mm.loc[outcome,'max_violation'] = max_viol
mm.loc[outcome, 'trust_region'] = True
mm.loc[outcome, 'enlargement'] = [1, 0, 0.1]
model_master = opticl.model_selection(mm, performance)

if not os.path.exists('experiments'):
    print('Creating folder...')
    os.makedirs('experiments')
model_master.to_csv('experiments/model_master_%s.csv' % (code_version), index = True)

opticl.check_model_master(model_master)
model_master

Checking model
No learned objective
1 learned constrained outcomes

Checking outcome palatability
Embedding outcome with single gbm model

Embedding constraint for palatability.
0.5 <= palatability


,model,task,objective,lb,ub,features,var_features,contex_features,group_models,group_method,ensemble_weights,max_violation,trust_region,dataset_path,clustering_model,enlargement,SCM_counterfactuals
palatability,{'results/gbm/vAAAI-23_WFPexample_palatability...,continuous,0,0.5,None,"Index(['Beans', 'Bulgur', 'Cheese', 'Fish', 'M...","Index(['Beans', 'Bulgur', 'Cheese', 'Fish', 'M...",{},False,violation,None,0.5,True,processed-data/WFP_dataset.csv,None,"[1, 0, 0.1]",None


In [25]:
start_cm = time.time()
conceptual_model= init_conceptual_model(cost_p)
cm = time.time() - start_cm

start_opticl = time.time()
model_TR = opticl.optimization_MIP(conceptual_model, model_master)
model_opticl = time.time() - start_opticl
# final_model.write('experiments/mip_%s_seed_%d.lp' % (code_version, seed))
opt = SolverFactory('glpk')
start_time = time.time()
results = opt.solve(model_TR) 
computation_time = time.time() - start_time
evaluation(model_TR, cm, model_opticl, computation_time)

Generating constraints for the trust region using 5000 samples.
The l1 norm is used for the enlarged CH trust region
The trust region is being enlarged with a constraint upper bounded by: 0.1.
... Trust region defined.
Embedding constraints for palatability
Adding single model.

################################### Summary ###################################
Optimal Solution: {'Beans': '17.34 g', 'Milk': '42.53 g', 'Salt': '5.0 g', 'Lentils': '30.45 g', 'Maize': '142.24 g', 'Sugar': '20.0 g', 'Oil': '21.81 g', 'Wheat': '221.66 g', 'WSB': '76.45 g'}

Total cost: 3314.860 $
Predicted palatability: 0.504
Real palatability: 0.623

Conceptual model defined in 0.008 seconds
Learned constraints & trust region embedded in 2.527 seconds
Problem Solved in 4.779 seconds
###############################################################################


## Robust Constraint Learning

There are two sources of uncertainty, and consequently notions of robustness, that can be considered when embedding a trained machine learning model as a constraint.  

**Function Uncertainty**. The first source of uncertainty is in the underlying functional form of $\hat{h}$. We do not know the ground truth relationship between $(\boldsymbol{x},\boldsymbol{w})$ and $y$, and there is potential for model mis-specification. We limit this risk through our nonparametric model selection procedure, namely training $\hat{h}$ for a diverse set of methods (e.g., decision tree, regression, neural network) and selecting the final model using a cross-validation procedure.

**Parameter Uncertainty**. Even within a single model class, there is uncertainty in the parameter estimates that define $\hat{h}$. Consider the case of linear regression. A regression estimator consists of point estimates of coefficients and an intercept term, but there is uncertainty in the estimates as they are derived from noisy data. We seek to make our model robust by using a model-wrapper approach, which is agnostic to the underlying model.

<br><br>
<font size="4">Model wrapper</font>

The model wrapper approach is agnostic to the underlying method. Rather than obtaining our estimated outcome from a single trained predictive model, we suppose that we have $P$ estimators. The set of estimators can be obtained by bootstrapping or by training models using different methods. The uncertainty is thus characterized by different realizations of the predicted value from multiple estimators.  

We introduce a constraint that at most $\alpha \in [0,1]$ proportion of the $P$ estimators violate the constraint. Let $\hat{h}_1,\ldots,\hat{h}_P$ be the individual estimators. Then $\hat{h}_i(x) \leq \tau$ in at least $1-\alpha P$ of these estimators. This allows for a degree of robustness to individual model predictions by \color{blue}{discarding a small number of potential outlier predictions.} Formally,
\begin{align}
    \frac{1}{P}\sum_{i=1}^P \mathbb{I} (y_i \leq \tau) \geq 1 - \alpha. \label{eqn:model_wrapper}
\end{align}
Note that $\alpha = 0$ enforces the bound for all estimators, yielding the most conservative estimate, whereas $\alpha = 1$ removes the constraint entirely. 

<font size="4">OptiCL Specs</font>  
- **group_method**: $average$ will constraint the average prediction to be $\leq \tau$. $violation$ will allow at most viol_rule (%) ML models to predict about the threshold $\tau$.
- **viol_rule**: percentage of violation.
- **bs**: number of ML models.
- **ensemble_weights**: weights of different ML models (*not supported yet*).

In [30]:
alg_dict = {'cart': None}
viol_rule = '0.2'

### Example with grouping/bootstrap
gr=True
bs = 5

print("Algorithms = %s" % alg_dict)
print("Bootstrap iterations = %d" % bs)
print("Violation rule = %s" % str(viol_rule))
code_version = 'AAAI-23_WFPexample'

version = 'vAAAI-23_WFPexample'
outcome = 'palatability'
threshold = 0.5

Algorithms = {'cart': None}
Bootstrap iterations = 5
Violation rule = 0.2


In [27]:
outcome_list = {'palatability': {'lb':threshold, 'ub':None, 'objective_weight':0,'group_models':gr,
'task_type': 'continuous', 'alg_list':alg_dict, 'bootstrap_iterations':bs,
                                   'X_train':X_train, 'y_train':y_train, 'X_test':X_test, 'y_test':y_test,
                                   'dataset_path':'processed-data/WFP_dataset.csv'}}

performance = opticl.train_ml_models(outcome_list, version)
if not os.path.exists('results'):
    os.makedirs('results')
performance.to_csv('results/%s_performance.csv' % (code_version))

Learning a model for palatability
Bootstrap iteration 1 of 5
training palatability_s0 with cart
------------- Initialize grid  ----------------
------------- Running model  ----------------
Algorithm = cart, metric = None
saving... results/cart_palatability_s0_trained.pkl
------------- Model evaluation  ----------------
-------------------training evaluation-----------------------
Train MSE: 0.011849332956652487
Train R2: 0.7596535401549386
-------------------testing evaluation-----------------------
Test MSE: 0.014127246086486515
Test R2: 0.7284120702429107

Bootstrap iteration 2 of 5
training palatability_s1 with cart
------------- Initialize grid  ----------------
------------- Running model  ----------------
Algorithm = cart, metric = None
saving... results/cart_palatability_s1_trained.pkl
------------- Model evaluation  ----------------
-------------------training evaluation-----------------------
Train MSE: 0.0117955192941538
Train R2: 0.7699757356309128
-------------------testin

In [31]:
columns_df = ['algorithm','iteration','price_matrix']+list(X.columns)+['objective_function', 'real_palat', 'pred_palat', 'violation', 'time']

print("\nPreparing model master")
gr_method = None
if viol_rule == 'average':
    gr_method = 'average'
    max_viol = None
    print("Group method = %s" % (gr_method))
    gr_string = 'average'
else: 
    gr_method = 'violation'
    max_viol = float(viol_rule)
    print("Group method = %s (violation limit = %.2f)" % (gr_method, max_viol))
    gr_string = 'violation_%.2f' % max_viol

mm = opticl.initialize_model_master(outcome_list)
mm.loc[outcome,'group_method'] = gr_method
mm.loc[outcome,'max_violation'] = max_viol
mm.loc[outcome, 'trust_region'] = True
model_master = opticl.model_selection(mm, performance)

if not os.path.exists('experiments'):
    print('Creating folder...')
    os.makedirs('experiments')
model_master.to_csv('experiments/model_master_%s.csv' % (code_version), index = True)

opticl.check_model_master(model_master)
model_master


Preparing model master
Group method = violation (violation limit = 0.20)
Checking model
No learned objective
1 learned constrained outcomes

Checking outcome palatability
Embedding outcome with ensemble of 5 models, aggregation = violation

Embedding constraint for palatability.
0.5 <= palatability


,model,task,objective,lb,ub,features,var_features,contex_features,group_models,group_method,ensemble_weights,max_violation,trust_region,dataset_path,clustering_model,enlargement,SCM_counterfactuals
palatability,{'results/cart/vAAAI-23_WFPexample_palatabilit...,continuous,0,0.5,None,"Index(['Beans', 'Bulgur', 'Cheese', 'Fish', 'M...","Index(['Beans', 'Bulgur', 'Cheese', 'Fish', 'M...",{},True,violation,None,0.2,True,processed-data/WFP_dataset.csv,None,[0],None


In [32]:
start_cm = time.time()
conceptual_model= init_conceptual_model(cost_p)
cm = time.time() - start_cm

start_opticl = time.time()
model_MW = opticl.optimization_MIP(conceptual_model, model_master)
model_opticl_time = time.time() - start_opticl
# final_model.write('experiments/mip_%s_seed_%d.lp' % (code_version, seed))
opt = SolverFactory('glpk')
start_time = time.time()
results = opt.solve(model_MW) 
computation_time = time.time() - start_time
evaluation(model_MW, cm, model_opticl_time, computation_time)

Generating constraints for the trust region using 5000 samples.
... Trust region defined.
Embedding constraints for palatability_0
Embedding constraints for palatability_1
Embedding constraints for palatability_2
Embedding constraints for palatability_3
Embedding constraints for palatability_4
palatability
Adding ensemble constraint with 5 models and violation limit = 0.20

################################### Summary ###################################
Optimal Solution: {'Beans': '48.15 g', 'DSM': '1.85 g', 'Milk': '47.05 g', 'Salt': '5.0 g', 'Lentils': '8.39 g', 'Maize': '50.74 g', 'Maize meal': '186.9 g', 'Soya-fortified bulgur wheat': '1.42 g', 'Sugar': '20.0 g', 'Oil': '26.02 g', 'Wheat': '105.78 g', 'WSB': '70.3 g'}

Total cost: 3511.343 $
Predicted palatability: 0.582
Real palatability: 0.683

Conceptual model defined in 0.005 seconds
Learned constraints & trust region embedded in 1.755 seconds
Problem Solved in 7.621 seconds
######################################################